# 🏉 Go long! Long range transformers

In recent research there has been an extensive study for improving the calculation of attention in transformer architectures. Mostly for improving their capacity to handle longer token sequences. 👊

The attention calculation is known to be quadratic in compatation time with respect to the sequence length 👎. These recent advances, however, are able to perform attention calculation in linear time with respect to the sequence length. This allows us to scale the transformer architecture such that it can handle input sequences beyond the usual token length of 512. 

In this notebook, we compare traditional transformers with novel efficient transformers. We'll use roBERTa as a baseline to compare against LongFormer and BigBird.  

Let's put these architectures to the test and see which one comes out on top 🏆!  


## 🛠️ Getting started: Install packages & download models

The below cells will setup everything that is required to get started with model training:

* Install python specific packages
* Import required packages

In [ ]:
!pip install -q sklearn transformers datasets torch plotly sentencepiece tqdm

In [2]:
import time 
import sys 
import json
import shutil
import pandas as pd
from enum import Enum
import math
import torch

import plotly.express as px
import plotly.graph_objects as go


from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BigBirdTokenizerFast, BigBirdForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, LongformerForSequenceClassification, TrainingArguments, Trainer, LongformerTokenizerFast
from datasets import load_dataset

## 💾 Dataset & downstream task

We will use the [Hyperpartisan news dataset](https://huggingface.co/datasets/hyperpartisan_news_detection) for binary sentiment classification. In the paper publication of LongFormer and BigBird, both architectures were compared against RoBERTa with this exact dataset.

This dataset contains on average wordpieces, which is ideal to make our point 💪.

We aim to gain more insight in when to use which architecture, therefore we will go *one step beyond 🔥*, and evaluate the architectures on distinct subsets of the data, each time introducing sentences with more tokens!

In [ ]:
# Load the tokenizer
tokenizer =  RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
# Load the hyperpartisan dataset
ds = load_dataset('hyperpartisan_news_detection', 'byarticle')['train']

# Rename the label column for uniformity
ds = ds.rename_column("hyperpartisan", "label")

# Remove unused columns
ds = ds.remove_columns(['title', 'url', 'published_at'])

# Add token length column to filter on later
ds=ds.add_column(name = 'token_length', column=[len(tokenizer.batch_encode_plus([x['text']]).input_ids[0]) for x in ds])

Split into train and test set

In [5]:
split_ds = ds.train_test_split(test_size=0.20)

Make various train partitions

In [ ]:
train_ds = split_ds['train']
test_ds = split_ds['test']

train_ds_dict = {}

for min_tok, max_tok in [(0,256), (0, 512), (0, 1024), (0, 2048), (0, 4096)]:
    # Filter on the lengths
    train_ds_dict[str(max_tok)] = train_ds.filter(lambda x : x['token_length'] <= max_tok).filter(lambda x : x['token_length'] > min_tok)

    # Select the closest even number
    # the longformer optimizer cannot handle a single remaining datapoint at the end of the epoch
    train_ds_dict[str(max_tok)] = train_ds_dict[str(max_tok)].select(range(
        math.floor(
            len(train_ds_dict[str(max_tok)]) / 2.
        ) * 2
    ))

# 💥 Models

Run the cells below to redo the training of each model

🛎️ Disclaimer: this will take some time... So if you're a busy bee or a hurrying hippo, you can skip this section and load in the results from one of our runs smoothly and swiftly 😊!

## 💪 Training

In [7]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def train_and_evaluate(timing_run, model, identifier, tokenizer, train_ds, test_ds, max_length=1024):
    def tokenization(batched_text):
        return tokenizer(batched_text['text'], padding='max_length', truncation=True, max_length=min(max_length, tokenizer.model_max_length))

    # tokenizing both the training and test dataset 
    train_ds = train_ds.map(tokenization,
                            batched=True,
                            batch_size=len(train_ds),
                            remove_columns=['text'])

    test_ds = test_ds.map(tokenization,
                          batched=True,
                          batch_size=len(test_ds),
                          remove_columns=['text'])

    train_ds.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    test_ds.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

    parameters = model.num_parameters()

    # Set different number of steps for accuracy and timing runs
    logging_steps = 1000 if timing_run else 10
    eval_steps = 1000 if timing_run else 10
    max_steps = 100 if timing_run else 250

    training_args = TrainingArguments(
        # Set the batch sizes
        per_device_train_batch_size=2,
        per_device_eval_batch_size=4,

        # Apply efficiency tricks
        gradient_accumulation_steps=8,
        fp16=True,
        
        # steps paramters
        evaluation_strategy="steps",
        warmup_steps=0,
        eval_steps=eval_steps, 
        max_steps=max_steps,
        logging_steps=logging_steps,

        # Optimizer parameters
        learning_rate=2e-5,

        # Finalization
        load_best_model_at_end=False if timing_run else True,
        metric_for_best_model='accuracy', # default value is validation loss, we want the model with the highest accuracy 
        
        # Output locations
        output_dir='./{}'.format(identifier),
        run_name='{}'.format(identifier),
        logging_dir='./{}-logging'.format(identifier),
        log_level='warning',
        save_strategy="steps"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_ds,
        eval_dataset=test_ds,
    )
    start_time = time.time()
    trainer.train()
    duration = time.time() - start_time

    shutil.rmtree('./{}'.format(identifier))

    metrics = None
    if not timing_run:
        metrics = trainer.evaluate()

    return trainer, parameters, metrics, duration

In [8]:
def run_training(timing_run, model, tokenizer, model_name, train_ds, test_ds, max_length=1024):
    torch.cuda.empty_cache()

    try:
        identifier = '{}-{}-{}'.format(model_name, max_length, 'timing' if timing_run else 'accuracy').lower()
        trainer, parameters, metrics, duration  = train_and_evaluate(timing_run, model, identifier, tokenizer, train_ds, test_ds, max_length)

        results_and_metrics = {
            "model_name": model_name,
            "parameters": parameters,
            "duration": duration,
            "metrics": metrics,
            'tokencount': str(max_length),
            'timing_run': timing_run
        }
        with open(f'{identifier}-results.json', "w") as fd:
            json.dump(results_and_metrics, fd)

    except RuntimeError:
        del trainer 

    del model 


### 🎯 Accuracy runs

In [9]:
timing = False
gradient_checkpointing = True

for size in [256, 512, 1024, 2048, 4096]:

    for model_name, tokenizer, model in [
        (
            'roberta',
            RobertaTokenizer.from_pretrained('roberta-base'),
            RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                             gradient_checkpointing=gradient_checkpointing,
                                                             num_labels=2)
        ),
        (
            'longformer',
            LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096'),
            LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                                gradient_checkpointing=gradient_checkpointing,
                                                                attention_window=128,
                                                                num_labels=2)
        ),
        (
            'bigbird',
            BigBirdTokenizerFast.from_pretrained('google/bigbird-roberta-base'),
            BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base',
                                                             gradient_checkpointing=gradient_checkpointing,
                                                             num_labels=2)
        )
        ]:

        print(f"Training {model_name} on size {size}")

        run_training(timing, model, tokenizer, model_name, train_ds_dict[str(size)], test_ds, max_length=size)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Training bigbird on size 4096


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,0.633300,0.663903,0.612403,0.000000,0.000000,0.000000
20,0.641200,0.606036,0.612403,0.000000,0.000000,0.000000
30,0.580800,0.628415,0.612403,0.000000,0.000000,0.000000
40,0.553400,0.494754,0.844961,0.772727,0.894737,0.680000
50,0.491300,0.455993,0.829457,0.780000,0.780000,0.780000
60,0.389800,0.453985,0.837209,0.740741,0.967742,0.600000
70,0.349100,0.388494,0.821705,0.747253,0.829268,0.680000
80,0.323500,0.404326,0.821705,0.788991,0.728814,0.860000
90,0.250600,0.378507,0.837209,0.778947,0.822222,0.740000
100,0.210200,0.392178,0.860465,0.808511,0.863636,0.760000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1310: FutureWarning:

Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F

### ⏱️ Timing runs

In [10]:
timing = True
gradient_checkpointing = True

for size in [256, 512, 1024, 2048, 4096]:

    for model_name, tokenizer, model in [
        (
            'roberta',
            RobertaTokenizer.from_pretrained('roberta-base'),
            RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                             gradient_checkpointing=gradient_checkpointing,
                                                             num_labels=2)
        ),
        (
            'longformer',
            LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096'),
            LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                                gradient_checkpointing=gradient_checkpointing,
                                                                attention_window=256,
                                                                num_labels=2)
        ),
        (
            'bigbird',
            BigBirdTokenizerFast.from_pretrained('google/bigbird-roberta-base'),
            BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base',
                                                             gradient_checkpointing=gradient_checkpointing,
                                                             num_labels=2)
        )
        ]:

        print(f"Training {model_name} on size {size}")

        run_training(timing, model, tokenizer, model_name, train_ds_dict[str(size)], test_ds, max_length=size)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Training roberta on size 256


Loading cached processed dataset at /root/.cache/huggingface/datasets/hyperpartisan_news_detection/byarticle/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-0687b0845c3b6c78.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Step,Training Loss,Validation Loss


...

Step,Training Loss,Validation Loss


Loading cached processed dataset at /root/.cache/huggingface/datasets/hyperpartisan_news_detection/byarticle/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-4a7990436ea74476.arrow


Training bigbird on size 4096


  0%|          | 0/1 [00:00<?, ?ba/s]

Step,Training Loss,Validation Loss


# 📈 Visualize

In [ ]:
import os 
run_dict = []

# loop to collect the results from each model
for root, dirs, files in os.walk('./'):
    for name in files:
        if name.endswith(("results.json")):
            full_path = os.path.join(root, name)
            with open(full_path) as f:
                data = json.load(f)
                run_dict.append(data)

In [12]:
df = pd.DataFrame(run_dict)
df2 = df.metrics.apply(pd.Series)
result = pd.concat([df.drop('metrics', axis=1), df2], axis=1)
result["iden"] = result["model_name"] + '-' + result["tokencount"].astype(str)
result['parameters'] /= 1e6
result.sort_values(by=['model_name', 'tokencount'], )

,model_name,parameters,duration,tokencount,timing_run,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch,iden
21,bigbird,128.060930,488.144646,1024,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bigbird-1024
22,bigbird,128.060930,1547.258527,1024,False,0.506037,0.852713,0.776471,0.942857,0.66,8.0108,16.103,4.119,13.15,bigbird-1024
9,bigbird,128.060930,925.536229,2048,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bigbird-2048
24,bigbird,128.060930,2830.359838,2048,False,0.465422,0.906977,0.866667,0.975000,0.78,15.7516,8.190,2.095,8.92,bigbird-2048
13,bigbird,128.060930,93.173873,256,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bigbird-256
23,bigbird,128.060930,449.907733,256,False,1.121046,0.736434,0.585366,0.750000,0.48,1.4328,90.036,23.032,83.30,bigbird-256
16,bigbird,128.060930,1819.089413,4096,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bigbird-4096
29,bigbird,128.060930,5416.983666,4096,False,0.347840,0.891473,0.851064,0.909091,0.80,29.4785,4.376,1.119,8.06,bigbird-4096
6,bigbird,128.060930,173.928012,512,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bigbird-512
7,bigbird,128.060930,633.653134,512,False,0.683401,0.837209,0.746988,0.939394,0.62,2.8548,45.187,11.560,27.71,bigbird-512


## Importing results (optional)

In [ ]:
!git clone https://github.com/ml6team/quick-tips.git
!cd quick-tips && git checkout feature/long_seq_transformers
!mv quick-tips/nlp/2021_06_29_long_range_transformers/results ./results
!rm -rf quick-tips

In [15]:
result = pd.read_csv("results/experiment_results.csv")

## Visualization

In [13]:
import plotly.graph_objects as go
sizes=['256', '512', '1024', '2048', '4096']

fig = go.Figure(data=[
    go.Bar(
        name=model,
        x=sizes,
        y=[result[
            (result['timing_run']==False) \
            & (result['model_name']==model)\
            & (result['tokencount']==size)]['eval_accuracy'].iloc[0]\
        for size in sizes]
    )
    for model in ['roberta', 'bigbird', 'longformer']
])

fig.update_layout(
    barmode='group',
    title="Accuracy of the various models and data token sizes",
    xaxis_title="Dataset token size",
    yaxis_title="Accuracy"
)

fig.show()

In [14]:
import plotly.graph_objects as go
sizes=['256', '512', '1024', '2048', '4096']

fig = go.Figure(data=[
    go.Bar(
        name=model,
        x=sizes,
        y=[result[
            (result['timing_run']==True) \
            & (result['model_name']==model)\
            & (result['tokencount']==size)]['duration'].iloc[0]\
        for size in sizes]
    )
    for model in ['roberta', 'bigbird', 'longformer']
])

fig.update_layout(
    barmode='group',
    title="Training time for 100 steps for the various models and dataset token sizes",
    xaxis_title="Dataset token size",
    yaxis_title="Training time (seconds)"
)

fig.show()

# 🎁 Wrapping up 

Nicely done! Let's wrap up some of our findings 🧪.

* Both LongFormer and BigBird outperform roBERTa for longer sequences by a small margin 💥 **Sparse attention for the win** 👑
* BigBird and LongFormer also show excellent performance on shorter sequences, in line with roBERTa's performance 💪 
* The **performance hit** between short and long sequence lengths is **less noticeable** for the efficient transformer architectures 
* LongFormer and BigBird have **higher training and inference times**. However, we didn't use the architectures to their full capactity of 4096 wordpieces 🙋‍

